In [25]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [46]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [55]:
# 규원님이 전처리 완료하신 최종 데이터 (+라벨까지 붙은)
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/data/dataset_210626_215600.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
len(df)

74779

In [56]:
df.head()

,gender,name,accords,review,tokenized,only_english,longer_than_2_A,stopwords_removed,lemmatizated,label
0,female,Alien Mugler for women,"['white floral', 'amber', 'woody']","Got a sample of this today, and my 9 year old ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'sample', 'this', 'today', 'and', 'yea...","['got', 'sample', 'today', 'year', 'old', 'dau...","['get', 'sample', 'today', 'year', 'old', 'dau...",2.0
1,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",First impressions. Test on blotter.\r\nI have ...,"['first', 'impressions.', 'test', 'on', 'blott...","['first', 'impressions', 'test', 'on', 'blotte...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impression', 'test', 'blotter', 'he...",2.0
2,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",This perfume reminds me of my best friend. Act...,"['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'best', 'friend...","['perfume', 'reminds', 'best', 'friend', 'actu...","['perfume', 'reminds', 'best', 'friend', 'actu...",2.0
3,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Imagine tripping over your own feet and fallin...,"['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'feet', 'falling', 'fa...","['imagine', 'trip', 'foot', 'fall', 'face', 'f...",2.0
4,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Gorgeous Gorgeous Blend ..\r\nLove the scent.....,"['gorgeous', 'gorgeous', 'blend', '..', 'love'...","['gorgeous', 'gorgeous', 'blend', '', 'love', ...","['gorgeous', 'gorgeous', 'blend', 'love', 'the...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...",2.0


본 데이터는 모두 string 형태라 전처리 필요

In [57]:
sent_text = df['lemmatizated']

In [58]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [59]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [60]:
for line in result[:3]:
  print(line)

['get', 'sample', 'today', 'year', 'old', 'daughter', 'think', 'smell', 'spray', 'card', 'rotten', 'fish', 'nearly', 'throw', 'immediately', 'however', 'later', 'even', 'decide', 'give', 'fair', 'trial', 'spray', 'crook', 'elbow', 'rot', 'fish', 'smell', 'time', 'get', 'definite', 'bit', 'funk', 'almost', 'urine', 'scent', 'minute', 'blossom', 'gorgeous', 'smooth', 'woody', 'ambery', 'clean', 'warm', 'jasmine', 'daughter', 'didnt', 'even', 'believe', 'tell', 'fragrance', 'smell', 'earlier', 'lol', 'immediately', 'comment', 'fresh', 'clean', 'vibe', 'agree', 'soapy', 'hint', 'powdery', 'way', 'obsess', 'cant', 'stop', 'smelling', 'arm', 'glad', 'give', 'try', 'wait', 'buy', 'full', 'bottle']
['first', 'impression', 'test', 'blotter', 'heard', 'smell', 'like', 'bag', 'spray', 'like', 'gabriella', 'sabatini', 'perfume', 'everybody', 'past', 'woman', 'wear', 'smell', 'addictive', 'plasticky', 'beautiful', 'amaze', 'projection', 'longevity', 'woman', 'strong', 'personality', 'wear', 'may', 

### Glove 훈련시키기
LSA의 메커니즘인 `카운트 기반 방법`과 Word2Vec의 메커니즘인 `예측 기반 방법` 모두 사용

In [6]:
! pip install glove-python-binary

     |████████████████████████████████| 952kB 4.7MB/s 


In [17]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(result, window=10)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=5, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 5.

Performing 5 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


### 5 Epoch 수행 후 결과

In [18]:
model_result = glove.most_similar("fruity")
print(model_result)

[('woody', 0.9618388002165104), ('slightly', 0.9589665439184879), ('citrusy', 0.9577111926091315), ('musky', 0.9569770976368478)]


In [19]:
model_result = glove.most_similar("aquatic")
print(model_result)

[('aromatic', 0.9581555232355182), ('citrusy', 0.9550930141741334), ('fruity', 0.9428428594594367), ('bright', 0.9418477974099341)]


In [20]:
model_result = glove.most_similar("tobacco")
print(model_result)

[('vanille', 0.9873953081179038), ('vanilla', 0.9435339652940836), ('incense', 0.9233757510251398), ('leather', 0.9209440959183094)]


In [21]:
model_result = glove.most_similar("floral")
print(model_result)

[('powdery', 0.9625309326771614), ('edge', 0.9570424986192864), ('musky', 0.9563490604299816), ('florals', 0.9552740324982202)]


In [22]:
model_result = glove.most_similar("orange")
print(model_result)

[('blossom', 0.9905494058142394), ('pear', 0.9704932799681479), ('ginger', 0.9667338806439641), ('grapefruit', 0.9657312627583805)]


### 모델 저장 및 로드

In [26]:
from gensim.models import Word2Vec, KeyedVectors
from glove import Corpus, Glove
# glove.save('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding/model/glove_10window_5epoch') # 모델 저장
loaded_model = Glove.load('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding/model/glove_10window_5epoch') # 모델 로드

In [27]:
model_result = loaded_model.most_similar("orange")
print(model_result)

[('blossom', 0.9905494058142394), ('pear', 0.9704932799681479), ('ginger', 0.9667338806439641), ('grapefruit', 0.9657312627583805)]


In [49]:
def word_similarity(model, word1, word2):
  a_word = model.word_vectors[model.dictionary[word1]]
  b_word = model.word_vectors[model.dictionary[word2]]
  cosine_dists = np.dot(a_word, b_word)/(norm(a_word)*norm(b_word))
  return cosine_dists

In [50]:
print(word_similarity(loaded_model,"ocean", "summer"))

0.5720446629630073


In [51]:
print(word_similarity(loaded_model,"forest", "woody"))

0.4573042805130869


In [53]:
print(word_similarity(loaded_model,"jasmine", "floral"))

0.8975129415989167


### 20 Epoch 수행

In [61]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(result, window=10)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 5.

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


### 20 Epoch 수행 후 결과

In [62]:
model_result = glove.most_similar("fruity")
print(model_result)

[('floral', 0.8322061992266294), ('flowery', 0.8176014224168885), ('tart', 0.8020830320338991), ('citrusy', 0.7963247031420364)]


In [63]:
model_result = glove.most_similar("aquatic")
print(model_result)

[('citrusy', 0.8588719206222856), ('fresh', 0.8450433151460736), ('citrus', 0.8318260137493171), ('watery', 0.8182088900441339)]


In [64]:
model_result = glove.most_similar("tobacco")
print(model_result)

[('vanille', 0.9308610273374298), ('pipe', 0.8533310366014046), ('chergui', 0.8422446802367194), ('hay', 0.8061315637956102)]


In [65]:
model_result = glove.most_similar("floral")
print(model_result)

[('florals', 0.8963224336487933), ('flowery', 0.8582244286297417), ('musky', 0.8368601277344946), ('fruity', 0.8322061992266298)]


In [66]:
model_result = glove.most_similar("orange")
print(model_result)

[('blossom', 0.9586058217370159), ('mandarin', 0.8861838945043912), ('peel', 0.8618614807209772), ('grapefruit', 0.7790902745155396)]


### 모델 저장 및 로드

In [68]:
from gensim.models import Word2Vec, KeyedVectors
from glove import Corpus, Glove
glove.save('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding/model/glove_10window_20epoch') # 모델 저장
loaded_model = Glove.load('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding/model/glove_10window_20epoch') # 모델 로드

In [70]:
model_result = loaded_model.most_similar("orange")
print(model_result)

[('blossom', 0.9586058217370159), ('mandarin', 0.8861838945043912), ('peel', 0.8618614807209772), ('grapefruit', 0.7790902745155396)]


In [71]:
def word_similarity(model, word1, word2):
  a_word = model.word_vectors[model.dictionary[word1]]
  b_word = model.word_vectors[model.dictionary[word2]]
  cosine_dists = np.dot(a_word, b_word)/(norm(a_word)*norm(b_word))
  return cosine_dists

In [72]:
print(word_similarity(loaded_model,"ocean", "summer"))

0.5792375277508008


In [73]:
print(word_similarity(loaded_model,"forest", "woody"))

0.37577734535018953


In [74]:
print(word_similarity(loaded_model,"jasmine", "floral"))

0.7322724518694862
